     NAMA  : MUHAMMAD IHSAN AL RIAWI
     NPM   : 193510482
     KELAS : IV E

In [1]:
import import_ipynb
from World import World
import numpy as np
import pandas as pd
import copy

importing Jupyter notebook from World.ipynb


In [2]:
def costruct_p(world, p = 0.8, step = -0.04):
    nstates = world.get_nstates()
    nrows = world.get_nrows()
    obsacle_index = world.get_stateobstacles()
    terminal_index = world.get_stateterminals()
    bad_index = obsacle_index + terminal_index
    reward = np.array([step] * 4 + [0] + [step] * 4 + [1, -1] + [step])
    actions = ["N", "S", "E", "W"]
    transition_models = {}
    for action in actions:
        transition_model = np.zeros((nstates, nstates))
        for i in range(1, nstates + 1):
            if i not in bad_index:
                if action == "N":
                    if i + nrows <= nstates and (i + nrows) not in obsacle_index:
                        transition_model[i - 1][i + nrows - 1] += (1 - p) / 2
                    else :
                        transition_model[i - 1][i - 1] += (1 - p) / 2
                    if 0 < i - nrows <= nstates and (i - nrows) not in obsacle_index:
                        transition_model[i - 1][i + nrows - 1] += (1 - p) / 2
                    else :
                        transition_model[i - 1][i - 1] += (1 - p) / 2
                    if (i - 1) % nrows > 0 and (i - 1) not in obsacle_index:
                        transition_model[i - 1][i - 1 - 1] += p
                    else :
                        transition_model[i - 1][i - 1] += p
                if action == "S":
                    if i + nrows <= nstates and (i + nrows) not in obsacle_index:
                        transition_model[i - 1][i + nrows - 1] += (1 - p) / 2
                    else :
                        transition_model[i - 1][i - 1] += (1 - p) / 2
                    if 0 < i - nrows <= nstates and (i - nrows) not in obsacle_index:
                        transition_model[i - 1][i + nrows - 1] += (1 - p) / 2
                    else :
                        transition_model[i - 1][i - 1] += (1 - p) / 2
                    if 0 < i % nrows and (i + 1) not in obsacle_index and (i + 1) <= nstates:
                        transition_model[i - 1][i + 1 - 1] += p
                    else :
                        transition_model[i - 1][i - 1] += p
                if action == "E":
                    if i + nrows <= nstates and (i + nrows) not in obsacle_index:
                        transition_model[i - 1][i - nrows - 1] += p
                    else :
                        transition_model[i - 1][i - 1] += p
                    if 0 < i % nrows and (i + 1) not in obsacle_index and (i + 1) <= nstates:
                        transition_model[i - 1][i + 1 - 1] += (1 - p) / 2
                    else :
                        transition_model[i - 1][i - 1] += (1 - p) / 2
                    if (i - 1) % nrows > 0 and (i - 1) not in obsacle_index:
                        transition_model[i - 1][i - 1 - 1] += (1 - p) / 2
                    else :
                        transition_model[i - 1][i - 1] += (1 - p) / 2
                if action == "W":
                    if 0 < i - nrows <= nstates and (i - nrows) not in obsacle_index:
                        transition_model[i - 1][i - nrows - 1] += p
                    else :
                        transition_model[i - 1][i - 1] += p
                    if 0 < i % nrows and (i + 1) not in obsacle_index and (i + 1) <= nstates:
                        transition_model[i + 1][i + 1 - 1] += (1 - p) / 2
                    else :
                        transition_model[i - 1][i - 1] += (1 - p) / 2
                    if (i - 1) % nrows > 0 and (i - 1) not in obsacle_index:
                        transition_model[i - 1][i - 1 - 1] += (1 - p) / 2
                    else :
                        transition_model[i - 1][i - 1] += (i - p) / 2
            elif i in terminal_index:
                transition_model[i - 1][i - 1] = 1
        transition_model[action] = pd.DataFrame(transition_model,
                                                index = range(1, nstates + 1), columns = range(1, nstates + 1))
    return transition_models, reward

In [3]:
def max_action(transition_models, reward, gamma, s, V, actions, terminal_ind):
    maxs = {key: 0 for key in actions}
    max_a = ""
    action_map = {k: v for k, v in zip(actions,[1, 2, 3, 4])}
    for action in actions:
        if s not in terminal_ind:
            maxs[action] += reward[s - 1] + gamma * np.dot(transition_models[action].loc[s, :].values, V)
        else :
            maxs[action] = reward[s - 1]
    maxi = -10 ** 10
    for key in maxs:
        if maxs[key] > maxi:
            max_a = key
            maxi = maxs[key]
    return maxi, action_map[max_a]

def value_iteration(world, transition_models, reward, gamma = 1.0, theta = 10 ** -4):
    nstates = world.get_nstates()
    terminal_ind = world.get_stateterminals
    V = np.zeros((nstates, ))
    P = np.zeros((nstates, 1))
    actions = ["N", "S", "E", "W"]
    delta = theta + 1
    while delta > theta:
        delta = 0
        v = copy.deepcopy(V)
        for s in range(1, nstates + 1):
            V[s - 1], P[s - 1] = max_action(transition_models, reward, gamma, s, v, actions, terminal_ind)
            delta = max(delta, np.abs(v[s - 1] - V[s - 1]))
    return V, P

def policy_iter(policy, world, transition_models, reward, gamma = 0.9, theta = 10 ** -4):
    nstates = world.get_nstates()
    terminal_ind = world.get_stateterminals()
    # initiate value function to zeros
    V = np.zeros((nstates, ))
    a = ["N", "S", "E", "W"]
    while True:
        delta = 0
        # for each state, perform a backup
        for s in range(nstates):
            v = 0
            # look at the policy actions and their probabilities
            for action, action_prob in enumerate(policy[s]):
                action = a[action]
                # for each action, calculate total gain
                if s not in terminal_ind:
                    v += rewards[s - 1] + action_prob * gamma * np.dot(transition_models[action].loc[s, :].values, V)
                else :
                    v = rewards[s - 1]
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
            print (V[s])
        # stop evaluating once the value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

def policy_improvement(world, transition_models, rewards, gamma = 0.9):
    nstates = world.get_nstates()
    nActions = world.getnactions()
    
    # start with a uniform policy
    policy = np.ones((nstates, nActions)) / nActions
    
    while True:
        # evaluate the current policy
        V = policy_iter(policy, world, transition_models, rewards, gamma)
        # will be set to false if we make any changes to the policy
        policy_stable = True
        
        for s in range(nstates):
            # the best action we would take under the current policy
            chosen_a = np.argmax(policy[s])
            action_values = lookfoword(s, V, transition_models, rewards, gamma)
            best_action = np.argmax(action_values)
            
            # greedily update the policy
            if chosen_a != best_action:
                policy_stable = False
            policy[s] = np.eye(nActions)[best_action]
        if policy_stable:
            return V, policy


In [4]:
if __name__ == "__main__":
    world = World()
    #world.plot()
    #world.plot_value([np.random.random() for i in range(12)])
    #world.plot_policy(np.random.randint(1, world.nActions, (world.nStates, 1)))
    # Part A
    # transition_models, rewards = construct_p(world)
    # Part B
    transition_models, reward = costruct_p(world)
    V, P = value_iteration(world, transition_models, rewards)
    world.plot_value(V)
    world.plot_policy(P)
    # Part C
    transition_models, rewards = costruct_p(world)
    V, P = value_iteration(world, transition_models, rewards, gamma = 0.9)
    world.plot_value(V)
    world.plot_policy(P)
    # Part D
    transition_models, rewards = costruct_p(world, step = -0.02)
    V, P = value_iteration(world, transition_models, rewards)
    world.plot_value(V)
    world.plot_policy(P)
    # Part E
    #transition_models, reward = construct_p(world)
    #V, P = value_iteration(world, transition_models, reward, gamma = 0.9)
    #world.plot_value(V)
    #world.plot_policy(P)

AttributeError: 'World' object has no attribute 'get_nstates'